In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from langchain.chat_models import init_chat_model

model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=.5)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=.5)
model_gpt_4o_mini = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=.5)

### Dynamic Models

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def state_based_model(request: ModelRequest, handler:Callable[[ModelRequest], ModelResponse])-> ModelResponse:
    """Select Model based on state conversation length"""
    message_count = len(request.state["messages"])

    if message_count > 10:
        #Long conversation - use model with larger context window
        model = model_gemini_flash
    else:
        model = model_llama_groq

    request = request.override(model=model)

    return handler(request)

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=model_llama_groq,
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

In [ ]:
print(response["messages"][-1].response_metadata["model_name"])

In [ ]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

In [ ]:
print(response["messages"][-1].response_metadata["model_name"])

### Dynamic Prompts

In [ ]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt,ModelRequest

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest)-> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} Only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [ ]:
agent = create_agent(
    model=model_llama_groq,
    middleware=[user_language_prompt],
    context_schema=LanguageContext
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Hindi")
)

print(response["messages"][-1].content)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="French")
)

print(response["messages"][-1].content)

### Dynamic Tools

In [ ]:
from langchain.tools import tool
from typing_extensions import Dict,Any
import sys
from pathlib import Path
# Assuming notebook is in notebooks/ folder
project_root = Path().resolve().parent  # adjust if needed
sys.path.append(str(project_root))
load_dotenv(override=True)

from utils.custom_tools import web_search
from utils.custom_tools import run_sql_query

#run_sql_query.invoke('Select count(*) from Genre')



'[(25,)]'

In [4]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role:str = 'external'



In [5]:

from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing_extensions import Callable

@wrap_model_call
def dynamic_tool_call(request:ModelRequest, handler:Callable[[ModelRequest], ModelResponse])->ModelResponse :

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role

    if user_role == 'internal':
        pass #internal users get access to all tools

    if user_role == 'external':
        tools = [web_search] #external users get access to only web search tool
        request = request.override(tools=tools)
    return handler(request)



In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=model_llama_groq,
    tools=[web_search, run_sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

370,062


In [8]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

How many artists are in the database?
================================== Ai Message ==================================
Tool Calls:
  web_search (wzh4b18mv)
 Call ID: wzh4b18mv
  Args:
    query: number of artists in database
================================= Tool Message =================================
Name: web_search

{"query": "number of artists in database", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.artnews.com/art-news/news/midjourney-ai-artists-database-1234691955/", "title": "Database of 16000 Artists Used to Train Midjourney AI ...", "content": "Artists included Warhol, Picasso, Cezanne, van Gogh, Anish Kapoor, Yayoi Kusama, Gerhard Richter, Frida Kahlo, and Banksy.", "score": 0.7300017, "raw_content": null}, {"url": "http://www.umdmusic.com/default.asp?Chart=D", "title": "Last Update: December 20, 2025 Total of: 370 062 artists", "content": "370

In [9]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

This means there are 275 artists in the database.


In [10]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

How many artists are in the database?
================================== Ai Message ==================================
Tool Calls:
  run_sql_query (z4pjk8tyb)
 Call ID: z4pjk8tyb
  Args:
    query: SELECT COUNT(*) FROM Artist;
================================= Tool Message =================================
Name: run_sql_query

[(275,)]
================================== Ai Message ==================================

This means there are 275 artists in the database.
